<h1 class="text-center">Unit tests: Basics of mocking in Python</h1>

<p class="text-center">by Joe Meilinger</p>

## Why mock?

- REDUCE EXTERNAL DEPENDENCIES!
    - Sometimes we can't rely on external services being up
    - Sometimes we don't wait to require a running database
- Prevent side effects
    - Did calling that service during our unit tests just cause us to hit a rate limit?
    - I don't really _want_ to send out 50 emails during my unit tests
- Reproduce corner-case situations
    - Fails on Tuesdays?!?!

## What should I use?

Would definitely recommend the built-in (Python 3.3+) `unittest.mock` library OR the backported version of that library.

However, there are a few other Python-based libraries:

- Flexmock: Ruby-inspired
- Fudge: Cool name

All other options are _really_ old...and these 2 options pale in comparison to `unittest.mock`.

## I'm still on Python 2.7...

Cool, no worries.

Just `pip install mock`

and you'll be good to go!

## Patching

Patching allows us to temporarily swap out the reference for a specific function/class to a mocked-version of it

In [ ]:
import time
from unittest import mock

def expensive_function():
    time.sleep(3)
    return 5

expensive_function()

In [ ]:
# Empty patch
with mock.patch('time.sleep') as f:
    print(expensive_function())

## What just happened?

- We patched the `time.sleep` function
    - Internally the reference to `time.sleep` was swaped out to a new blank instance of a `MagicMock`
    - The created MagicMock responded to the function call
    - Reference was restored as soon as we left the context block

In [ ]:
with mock.patch('time.sleep') as f:
    print(f)
    print(expensive_function())

# Proof that the original reference to `time.sleep` was restored...
print('This call takes the expected 3 seconds: {}'.format(expensive_function()))

## MagicMocks are awesome!

- Allow you to shim pretty much anything you want

In [ ]:
from unittest.mock import MagicMock

m = MagicMock()

m.dummy_function.return_value = 'this is a mocked function'
m.dummy_attr = 'dummy attribute'

print('Default call on mock', m())
print('m.dummy_function.return_value:', m.dummy_function())

m.return_value.color = 'yellow'

print('Mocked return object "color" attribute:', m().color)

## Side effects

- Sometimes you want to ensure that an exception will be handled correctly by your calling code, exception injection
- If you want subsequent calls to return DIFFERENT values you'll need to construct a new mock or use side effects

In [ ]:
def external_task():
    # Function performs a task that may/may not throw an exception
    return True

def regular_function():
    return external_task()

with mock.patch('__main__.external_task') as m:
    m.side_effect = Exception('Handle THIS!')
    
    external_task()

## Side effects (mock effect ordering)

In [ ]:
import random

# NOTE: Relying on state in this manner is bad

def get_number():
    return random.choice([1, 2, 3, 4])

def function_with_state_dependency():
    if get_number() < 3:
        return True
    else:
        return False

with mock.patch('__main__.get_number') as f:
    f.side_effect = [1, 3]
    
    print('First call:', function_with_state_dependency())
    print('Second call:', function_with_state_dependency())

## Use case: geocoding

- We don't want to eat up geocoding API limits during unit tests
- If we don't mock this, we'll have a external dependency tied to our application

In [ ]:
import geocoder
from unittest import mock, TestCase, TestSuite, TestLoader, TextTestRunner


class User:
    def __init__(self, address='20 S. Sarah Street, St. Louis, MO'):
        self.location = geocoder.google(address).latlng

        
class Tests(TestCase):
    @mock.patch('geocoder.google')
    def test_create_user(self, mock_geocoder):
        mock_geocoder.return_value.latlng = [38.6362566, -90.2466767]
        
        u = User()
        # Ok, we just spent an API credit if we didn't mock this out
        self.assertAlmostEqual(u.location[0], 38.6, places=1)
        self.assertAlmostEqual(u.location[1], -90.2, places=1)
        
        # Can also assert that specific calls to mock are only execute a certain number of times #awesome
        mock_geocoder.assert_called_once()
    
# Try w/o tests
u = User()
print('User location:', u.location)

# Try using mocked tests
suite = TestLoader().loadTestsFromTestCase(Tests)
runner = TextTestRunner()
runner.run(suite)

## Use case: database

- Don't necessarily always want to rely on a database connection when running unit tests

In [ ]:
import psycopg2
from psycopg2.extras import DictCursor
from unittest import mock, TestCase, TestSuite, TestLoader, TextTestRunner

conn = psycopg2.connect(dbname='mydatabase')

with conn.cursor() as c:
    c.execute('drop table if exists transactions')
    c.execute('create table transactions (id serial, amount money);')
    c.execute('insert into transactions (amount) values (1.50), (2), (-3), (82.25);')
    conn.commit()

# Sum of transactions = $82.75

class TransactionCounter:
    @staticmethod
    def count_transactions(conn):
        c = conn.cursor()
        c.execute('select sum(amount) from transactions;')
        return c.fetchall()[0][0]


class Tests(TestCase):
    @mock.patch('__main__.conn')
    def test_transaction_sum(self, mock_connection):
        mock_cur = mock_connection.cursor.return_value
        query_result = [('$82.75',)]
        mock_cur.fetchall.return_value = query_result
        
        self.assertEqual(TransactionCounter.count_transactions(mock_connection), '$82.75')

# Living DB required
print(TransactionCounter.count_transactions(conn))
        
# No DB required when mocked   
suite = TestLoader().loadTestsFromTestCase(Tests)
runner = TextTestRunner()
runner.run(suite)        

conn.close()

## Where to go from here?

- Speccing magic mocks
- Mocking full classes
- https://mockaroo.com/

<h1 class="text-center">Thank you!</h1>

<h1 class="text-center">Questions?</h1>

<p class="text-center">also feel free to hit me up at <a href='mailto:joemeilinger@gmail.com'>joemeilinger@gmail.com</a> any time or @meilinger on stltech.org slack</p>